In [2]:
# Import des libraries

from bigml.api import BigML
import pandas
from pandas import read_csv
import matplotlib

In [3]:
# Modification du fichier Train de Kaggle

df = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-train.csv', index_col=0)
df = df.fillna(0)
df['MonthlyPerPerson'] = df['MonthlyIncome'] / ( df['NumberOfDependents'] + 1 )
df.loc[df.age > 80, 'isOld'] = '1' 
df.loc[df.age <= 80, 'isOld'] = '0' 
df.to_csv('train_full.csv')

In [5]:
# Connexion au projet via l'API de BIG ML

api = BigML('MAXIMEFAVOT', '69757fb108f0bd7a8678e0b66e3de66ae6478c81', project='project/5d9e081659f5c320e1000597')

In [6]:
# Creation des sources sur BigML

source_train = api.create_source('train_full.csv')
trainfull_dataset = api.create_dataset(source_train)

In [7]:
# Création du split 80/20

train_dataset = api.create_dataset(
    trainfull_dataset, {"name": "Training_split", "sample_rate": 0.8, "seed": "my seed"})
test_dataset = api.create_dataset(
    trainfull_dataset, {"name": "Test_split", "sample_rate": 0.8, "seed": "my seed", "out_of_bag": True})

In [8]:
# Création du modele Ensemble

ensemble = api.create_ensemble(train_dataset, {"name": "ensemble", "objective_field": "SeriousDlqin2yrs"})

In [9]:
# Batch Prediction et Download

batch_prediction = api.create_batch_prediction(ensemble, test_dataset,{"all_fields": True, "probabilities": True})

In [11]:
api.download_batch_prediction(batch_prediction, filename='Prediction2.csv')

'Prediction2.csv'

In [13]:
evaluation = api.create_evaluation(ensemble, test_dataset)
api.ok(evaluation)
api.pprint(evaluation['object']['result']['model']['accuracy'])
api.pprint(evaluation['object']['result']['model']['average_area_under_roc_curve'])
api.pprint(evaluation['object']['result']['model']['confusion_matrix'])

0.93557
0.85479
[[27711, 274], [1659, 356]]
